In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
class Continuous_gridWorld:
    def __init__(self, blocked_center):
        self.yaxis = 100
        self.xaxis = 100
        self.blocked_center = blocked_center
        self.initial_state = np.array([(3, 3), (3, 4), (3, 5), (4, 3), (4, 4), (4, 5), (5, 3), (5, 4), (5, 5)])
    
    def create_blocked(self):
        self.blocked_list = []
        for i in self.blocked_center:
            for j in range(i[0]-6, i[0] + 5):
                for k in range(i[1] - 6, i[1] + 5):
                    if k < self.xaxis or j < self.yaxis or k >= 0 or j >= 0:
                        if k not in self.blocked_list:
                            self.blocked_list.append((j,k))
        return self.blocked_list  
    
    def show_world(self):
        grid = np.zeros([self.xaxis, self.yaxis])
        for i in self.blocked_list:
            grid[i[0]][i[1]] = 1
        return grid
    
    def cost_movement(self, state):
        for i in state:
            if i in self.blocked_list:
                cost = 1000
            else:
                cost = 1
        return cost
    
    def reset(self):
        self.state = self.initial_state
        return self.state
    
    def transition_function(self, action):
        if action == 'L':
            nxtState = [(s[0] - 10 , s[1]) for s in self.state if s[0] - 10 >= 0]
        elif action == 'R':
            nxtState = [(s[0] + 10 , s[1]) for s in self.state if s[0] + 10 <= self.xaxis - 1]
        elif action == 'U':
            nxtState = [(s[0], s[1] + 10) for s in self.state if s[1] + 10 <= self.yaxis - 1]
        else:
            nxtState = [(s[0], s[1] - 10) for s in self.state if s[1] - 10 >= 0]
        
        for s in self.state:
            if (s[0] >= 0) and (s[0] <= self.xaxis - 1):
                if (s[1] >= 0) and (s[1] <= self.yaxis - 1):
                    return nxtState
        return self.state
    
    def terminal_state(self):
        terminal_state = []
        for i in range(20):
            for j in range(20):
                terminal_state.append([i,j])
        return terminal_state
    
class Agent:
    def __init__(self, gamma, alpha):
        self.gamma = gamma
        self.alpha = alpha
        world = Continuous_gridWorld()
        self.actions = ['L', 'R', 'U', 'D']
        
        self.qtable = []
        for q in range(0, 10):
            self.qtable.append([])
            for p in range(0, 10):
                self.qtable[q].append([])
                for a in range(4):
                    self.qtable[q][p].append(0)
     
    def action_selection(self, epsilon, state):
        if random.random <= epsilon:
            action = random.randrange(self.actions)
        else:
            action_index = np.argmin(self.qtable[state[0]//10][state[1]//10])
            action = self.actions[action_index]
        return action
    
    def q_value_update(self, currentState, nxtState, action, cost):
        min_nxtState = np.argmin(self.qtable[nxtState[0]//10][nxtState[1]//10])
        self.qtable[currentState[0]//10][currentState[1]//10][action] += self.alpha * (
            cost + self.gamma * min_nxtState - self.qtable[currentState[0]//10][currentState[1]//10][action])
    
    def play(self, alpha = 0.9, gamma = 0.2, episode = 500):
        for z in range(epsiode):
            

In [3]:
x = Continuous_gridWorld([(50, 50), (60, 50), (70, 50), (90, 50), (60,65), (60, 75), (60, 85), (60, 95)] )
y = x.create_blocked()
# print(y[0])
z = x.show_world()
print(z)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
